In [ ]:
import operator
from pathlib import Path
from pprint import pprint
import requests

In [ ]:
URL = 'https://api.github.com'
USER = 'bjlittle'
REPO = 'dash'

In [ ]:
def api_token():
    fname = Path('token.txt')
    if not fname.exists():
        raise IOError("Problem with {}".fname.name)
    token = fname.read_text().strip()
    return token

In [ ]:
def get_content(target, accept=None):
    if accept is None:
        accept = 'application/vnd.github.v3+json'
    token = api_token()
    headers = {'Accept': accept, 'Authorization': 'token {}'.format(token)}
    response = requests.get('{}{}'.format(URL, target), headers=headers)
    response.raise_for_status()
    content = response.json()
    return content

In [ ]:
def post_content(target, json, accept=None):
    if accept is None:
        accept = 'application/vnd.github.v3+json'
    token = api_token()
    headers = {'Accept': accept, 'Authorization': 'token {}'.format(token)}
    response = requests.post('{}{}'.format(URL, target), json=json, headers=headers)
    response.raise_for_status()
    content = response.json()
    return content

In [ ]:
def get_issues(user, repo, label=None):
    target = '/repos/{}/{}/issues'.format(user, repo)
    issues = get_content(target)
    result = []
    for issue in issues:
        if label is not None:
            for item in issue['labels']:
                if item['name'].lower() == label.lower():
                    result.append(issue)
        else:
            result.append(issue)
    return result

In [ ]:
def get_projects(user, repo, label=None):
    target = '/repos/{}/{}/projects'.format(user, repo)
    projects = get_content(target, accept='application/vnd.github.inertia-preview+json')
    if label is not None:
        name = '{} project'.format(label.lower())
        for project in projects:
            if project['name'].lower() == name:
                result = project
                break
    else:
        result = projects
    return result

In [ ]:
def get_project_columns(project, name=None):
    target = '/projects/{}/columns'.format(project['id'])
    columns = get_content(target, accept='application/vnd.github.inertia-preview+json')
    if name is not None:
        for column in columns:
            if column['name'].lower() == name.lower():
                result = column
                break
    else:
        result = columns
    return result

In [ ]:
def get_project_column_cards(column):
    target = '/projects/columns/{}/cards'.format(column['id'])
    cards = get_content(target, accept='application/vnd.github.inertia-preview+json')
    return cards

In [ ]:
def post_project_column_cards(column, issues):
    target = '/projects/columns/{}/cards'.format(column['id'])
    for issue in issues:
        json = {'content_id': issue['id'], 'content_type': 'Issue'}
        response = post_content(target, json, accept='application/vnd.github.inertia-preview+json')

In [ ]:
def get_project_issue_urls(project):
    columns = get_project_columns(project)
    urls = []
    for column in columns:
        for card in get_project_column_cards(column):
            urls.append(card['content_url'])
    return urls

In [ ]:
def filter_project_issues(project, issues):
    urls = get_project_issue_urls(project)
    result = []
    for issue in issues:
        if issue['url'] not in urls:
            result.append(issue)
    return result

In [ ]:
label = 'bug'
issues = get_issues(USER, REPO, label=label)
project = get_projects(USER, REPO, label=label)
column = get_project_columns(project, name='backlog')
issues = filter_project_issues(project, issues)
post_project_column_cards(column, issues)